# Integrated Test
In this test, we'll use a SageMaker Estimator (https://sagemaker.readthedocs.io/en/stable/estimators.html) to encapsulate the docker image published to ECR and start a **local** test, but this time, using the SageMaker library.

In [1]:
import sagemaker
import json
from sagemaker import get_execution_role

role = get_execution_role()
sagemaker_session = sagemaker.Session()
bucket = sagemaker_session.default_bucket()
prefix='mlops/iris'

## Upload the dataset
In the previous exercise, prepared the training and validation dataset. Now, we'll upload the CSVs to S3 and share them with an Estimator

In [2]:
train_path = sagemaker_session.upload_data(path='input/data/train', key_prefix='iris-model/input/train')
test_path = sagemaker_session.upload_data(path='input/data/validation', key_prefix='iris-model/input/validation')
print("Train: %s\nValidation: %s" % (train_path, test_path) )

Train: s3://sagemaker-us-east-1-757420736997/iris-model/input/train
Validation: s3://sagemaker-us-east-1-757420736997/iris-model/input/validation


## And now, we can use a SageMaker Estimator for training and deploying the container we've created

In [3]:
# Create the estimator
# iris-model:test is the name of the container created in the previous notebook
# By the local codebuild test. An image with that name:tag was pushed to the ECR.
iris = sagemaker.estimator.Estimator('iris-model:test',
                                    role,
                                    instance_count=1, 
                                    instance_type='local',
                                    output_path='s3://{}/{}/output'.format(bucket, prefix))
hyperparameters = {
    'max_depth': 20,
    'n_jobs': 4,
    'n_estimators': 120
}

print(hyperparameters)
iris.set_hyperparameters(**hyperparameters)

{'max_depth': 20, 'n_jobs': 4, 'n_estimators': 120}


After you call .fit, a new training job will be executed inside the *local Docker daemon* and not in the SageMaker environment, on the cloud

In [4]:
iris.fit({'train': train_path, 'validation': test_path })

Creating tmpbf6qmkni_algo-1-cfsqy_1 ... 
Attaching to tmpbf6qmkni_algo-1-cfsqy_12mdone
algo-1-cfsqy_1  | Training mode
algo-1-cfsqy_1  | Training the classifier
algo-1-cfsqy_1  | [Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
algo-1-cfsqy_1  | [Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.1s
algo-1-cfsqy_1  | [Parallel(n_jobs=4)]: Done 120 out of 120 | elapsed:    0.2s finished
algo-1-cfsqy_1  | [Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
algo-1-cfsqy_1  | [Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
algo-1-cfsqy_1  | [Parallel(n_jobs=4)]: Done 120 out of 120 | elapsed:    0.0s finished
algo-1-cfsqy_1  | Score: 0.98
tmpbf6qmkni_algo-1-cfsqy_1 exited with code 0
Aborting on container exit...
===== Job Complete =====


The next command will launch a new container in your local Docker daemon. Then you can use the returned predictor for testing it

In [5]:
iris_predictor = iris.deploy(initial_instance_count=1, instance_type='local')

Attaching to tmpeuo9basq_algo-1-6926j_1
algo-1-6926j_1  | 2020-12-15 01:57:41,802 [INFO ] main com.amazonaws.ml.mms.ModelServer - 
algo-1-6926j_1  | MMS Home: /usr/local/lib/python3.7/site-packages
algo-1-6926j_1  | Current directory: /
algo-1-6926j_1  | Temp directory: /tmp
algo-1-6926j_1  | Number of GPUs: 0
algo-1-6926j_1  | Number of CPUs: 4
algo-1-6926j_1  | Max heap size: 4012 M
algo-1-6926j_1  | Python executable: /usr/local/bin/python
algo-1-6926j_1  | Config file: /etc/sagemaker-mms.properties
algo-1-6926j_1  | Inference address: http://0.0.0.0:8080
algo-1-6926j_1  | Management address: http://0.0.0.0:8080
algo-1-6926j_1  | Model Store: /.sagemaker/mms/models
algo-1-6926j_1  | Initial Models: ALL
algo-1-6926j_1  | Log dir: /logs
algo-1-6926j_1  | Metrics dir: /logs
algo-1-6926j_1  | Netty threads: 0
algo-1-6926j_1  | Netty client threads: 0
algo-1-6926j_1  | Default workers per model: 4
algo-1-6926j_1  | Blacklist Regex: N/A
algo-1-6926j_1  | Maximum Response Size: 6553500
alg

Now, let's use the predictor (https://sagemaker.readthedocs.io/en/stable/predictors.html) for some tests.

In [6]:
import pandas as pd
import random
from sagemaker.serializers import CSVSerializer
from sagemaker.deserializers import CSVDeserializer

# configure the predictor to do everything for us
iris_predictor.serializer = CSVSerializer()
iris_predictor.deserializer = CSVDeserializer()

# load the testing data from the validation csv
validation = pd.read_csv('input/data/validation/testing.csv', header=None)
idx = random.randint(0,len(validation)-5)
req = validation.iloc[idx:idx+5].values

# cut a sample with 5 lines from our dataset and then split the label from the features.
X = req[:,1:].tolist()
y = req[:,0].tolist()

# call the local endpoint
for features,label in zip(X,y):
    prediction = iris_predictor.predict(features)

    # compare the results
    print("RESULT: {} == {} ? {}".format( label, prediction, label == prediction ) )

algo-1-6926j_1  | 2020-12-15 01:57:43,862 [WARN ] W-model-2-stderr com.amazonaws.ml.mms.wlm.WorkerLifeCycle - [Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
algo-1-6926j_1  | 2020-12-15 01:57:43,874 [WARN ] W-model-2-stderr com.amazonaws.ml.mms.wlm.WorkerLifeCycle - [Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
algo-1-6926j_1  | 2020-12-15 01:57:43,886 [WARN ] W-model-2-stderr com.amazonaws.ml.mms.wlm.WorkerLifeCycle - [Parallel(n_jobs=4)]: Done 120 out of 120 | elapsed:    0.0s finished
algo-1-6926j_1  | 2020-12-15 01:57:43,967 [INFO ] W-9000-model com.amazonaws.ml.mms.wlm.WorkerThread - Backend response time: 107
algo-1-6926j_1  | 2020-12-15 01:57:43,968 [INFO ] W-9000-model ACCESS_LOG - /172.18.0.1:49058 "POST /invocations HTTP/1.1" 200 111
RESULT: 1.0 == [['1.0']] ? False
algo-1-6926j_1  | 2020-12-15 01:57:43,974 [WARN ] W-model-1-stderr com.amazonaws.ml.mms.wlm.WorkerLifeCycle - [Parallel(n_jobs=4)]: Using backend ThreadingBackend wi

In [7]:
iris_predictor.delete_endpoint()

Gracefully stopping... (press Ctrl+C again to force)


### That's it! :) Now you can go back to the previous Jupyter notebook and commit the assets to start building the Final Docker Image